https://udlbook.github.io/udlbook/

https://github.com/udlbook/udlbook/blob/main/Notebooks/Chap07/7_1_Backpropagation_in_Toy_Model.ipynb


# **Блокнот 7.1: Обратное распространение (backpropagation) в упрощенной модели (toy model)**

В этом блокноте вычисляются производные упрощенной функции (toy function), описанной в разделе 7.3 книги.

Пройдитесь по ячейкам ниже, запуская каждую ячейку по очереди. В разных местах вы увидите метку "TODO". Следуйте инструкциям в этих местах и сделайте прогнозы о том, что должно произойти, или напишите код для выполнения функций.

Мы собираемся исследовать, как брать производные композиций функций.  Для примера рассмотрим модель:

\begin{equation}
     \text{f}[x,\boldsymbol\phi] = \beta_3+\omega_3\cdot\cos\Bigl[\beta_2+\omega_2\cdot\exp\bigl[\beta_1+\omega_1\cdot\sin[\beta_0+\omega_0x]\bigr]\Bigr],
\end{equation}


с параметрами $\boldsymbol\phi=\{\beta_0,\omega_0,\beta_1,\omega_1,\beta_2,\omega_2,\beta_3,\omega_3\}$.<br>


Это композиция функций $\cos[\bullet],\exp[\bullet],\sin[\bullet]$.   Эти функции выбраны только потому, что вы, вероятно, уже знаете производные от этих функций:

\begin{align}
 \frac{\partial \cos[z]}{\partial z} = -\sin[z] \quad\quad \frac{\partial \exp[z]}{\partial z} = \exp[z] \quad\quad \frac{\partial \sin[z]}{\partial z} = \cos[z].
\end{align}


Предположим, что у нас есть квадратичная функция потерь:

\begin{equation*}
\ell_i = (\text{f}[x_i,\boldsymbol\phi]-y_i)^2,
\end{equation*}


Предположим, что мы знаем текущие значения $\beta_{0},\beta_{1},\beta_{2},\beta_{3},\omega_{0},\omega_{1},\omega_{2},\omega_{3}$, $x_i$ и $y_i$. Очевидно, мы могли бы вычислить $\ell_i$.   Но мы также хотим знать, как меняется $\ell_i$, когда мы вносим небольшое изменение в $\beta_{0},\beta_{1},\beta_{2},\beta_{3},\omega_{0},\omega_{1},\omega_{2}$, или $\omega_{3}$.  Другими словами, мы хотим вычислить восемь производных:

\begin{align}
\frac{\partial \ell_i}{\partial \beta_{0}}, \quad \frac{\partial \ell_i}{\partial \beta_{1}}, \quad \frac{\partial \ell_i}{\partial \beta_{2}}, \quad \frac{\partial \ell_i }{\partial \beta_{3}},  \quad \frac{\partial \ell_i}{\partial \omega_{0}}, \quad \frac{\partial \ell_i}{\partial \omega_{1}}, \quad \frac{\partial \ell_i}{\partial \omega_{2}},  \quad\text{и} \quad \frac{\partial \ell_i}{\partial \omega_{3}}.
\end{align}




In [2]:
# импорт библиотеки
import numpy as np

Давайте сначала определим исходную функцию для $y$ и функцию потерь:

In [3]:
def fn(x, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3):
  return beta3+omega3 * np.cos(beta2 + omega2 * np.exp(beta1 + omega1 * np.sin(beta0 + omega0 * x)))

def loss(x, y, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3):
  diff = fn(x, beta0, beta1, beta2, beta3, omega0, omega1, omega2, omega3) - y
  return diff * diff

Теперь выберем некоторые значения для beta, omega и x и вычислим выходное значение функции:

In [4]:
beta0 = 1.0; beta1 = 2.0; beta2 = -3.0; beta3 = 0.4
omega0 = 0.1; omega1 = -0.4; omega2 = 2.0; omega3 = 3.0
x = 2.3; y =2.0
l_i_func = loss(x,y,beta0,beta1,beta2,beta3,omega0,omega1,omega2,omega3)
print('l_i=%3.3f'%l_i_func)

l_i=0.139


# Вычисление производных вручную

Мы могли бы вычислить выражения для производных вручную и написать код для их непосредственного вычисления, но некоторые имеют очень сложные выражения даже для этого относительно простого исходного уравнения. Например:

\begin{align}
\frac{\partial \ell_i}{\partial \omega_{0}} &=& -2 \left( \beta_3+\omega_3\cdot\cos\Bigl[\beta_2+\omega_2\cdot\exp\bigl[\beta_1+\omega_1\cdot\sin[\beta_0+\omega_0\cdot x_i]\bigr]\Bigr]-y_i\right)\nonumber \\
&&\hspace{0.5cm}\cdot \omega_1\omega_2\omega_3\cdot x_i\cdot\cos[\beta_0+\omega_0 \cdot x_i]\cdot\exp\Bigl[\beta_1 + \omega_1 \cdot \sin[\beta_0+\omega_0\cdot x_i]\Bigr]\nonumber\\
&& \hspace{1cm}\cdot \sin\biggl[\beta_2+\omega_2\cdot \exp\Bigl[\beta_1 + \omega_1 \cdot \sin[\beta_0+\omega_0\cdot x_i]\Bigr]\biggr].
\end{align}

In [5]:
dldbeta3_func = 2 * (beta3 +omega3 * np.cos(beta2 + omega2 * np.exp(beta1+omega1 * np.sin(beta0+omega0 * x)))-y)
dldomega0_func = -2 *(beta3 +omega3 * np.cos(beta2 + omega2 * np.exp(beta1+omega1 * np.sin(beta0+omega0 * x)))-y) * \
              omega1 * omega2 * omega3 * x * np.cos(beta0 + omega0 * x) * np.exp(beta1 +omega1 * np.sin(beta0 + omega0 * x)) *\
              np.sin(beta2 + omega2 * np.exp(beta1+ omega1* np.sin(beta0+omega0 * x)))

Давайте убедимся, что расчет верен, используя конечные разности:

In [6]:
dldomega0_fd = (loss(x,y,beta0,beta1,beta2,beta3,omega0+0.00001,omega1,omega2,omega3)-loss(x,y,beta0,beta1,beta2,beta3,omega0,omega1,omega2,omega3))/0.00001

print('dydomega0: Function value = %3.3f, Finite difference value = %3.3f'%(dldomega0_func,dldomega0_fd))

dydomega0: Function value = 5.246, Finite difference value = 5.246




Код для вычисления $\partial l_i/ \partial \omega_0$ немного похож на кошмар. Легко допустить ошибки, и вы можете видеть, что некоторые его части повторяются (например, член $\sin[\bullet]$), что предполагает некоторую избыточность в вычислениях. Цель этого практического занятия - вычислить производные гораздо более простым способом. Расчет предполагает три шага:

**Шаг 1:** Запишем исходные уравнения в виде серии промежуточных вычислений.

\begin{align}
f_{0} &=& \beta_{0} + \omega_{0} x_i\nonumber\\
h_{1} &=& \sin[f_{0}]\nonumber\\
f_{1} &=& \beta_{1} + \omega_{1}h_{1}\nonumber\\
h_{2} &=& \exp[f_{1}]\nonumber\\
f_{2} &=& \beta_{2} + \omega_{2} h_{2}\nonumber\\
h_{3} &=& \cos[f_{2}]\nonumber\\
f_{3} &=& \beta_{3} + \omega_{3}h_{3}\nonumber\\
l_i &=& (f_3-y_i)^2
\end{align}


и вычислим и сохраним значения всех этих промежуточных значений.  Они понадобятся нам для вычисления производных.<br> Это называется **прямой проход** (**forward pass**).

In [7]:
# TODO вычислите все члены f_k и h_k
# Замените приведенный ниже код
f0 = beta0 + omega0*x
h1 = np.sin(f0)
f1 = beta1 + omega1*h1
h2 = np.exp(f1)
f2 = beta2 + omega2*h2
h3 = np.cos(f2)
f3 = beta3 + omega3*h3
l_i = (f3 - y)**2


In [8]:
# Давайте проверим, все ли мы поняли правильно:
print("f0: true value = %3.3f, your value = %3.3f"%(1.230, f0))
print("h1: true value = %3.3f, your value = %3.3f"%(0.942, h1))
print("f1: true value = %3.3f, your value = %3.3f"%(1.623, f1))
print("h2: true value = %3.3f, your value = %3.3f"%(5.068, h2))
print("f2: true value = %3.3f, your value = %3.3f"%(7.137, f2))
print("h3: true value = %3.3f, your value = %3.3f"%(0.657, h3))
print("f3: true value = %3.3f, your value = %3.3f"%(2.372, f3))
print("like original = %3.3f, like from forward pass = %3.3f"%(l_i_func, l_i))


f0: true value = 1.230, your value = 1.230
h1: true value = 0.942, your value = 0.942
f1: true value = 1.623, your value = 1.623
h2: true value = 5.068, your value = 5.068
f2: true value = 7.137, your value = 7.137
h3: true value = 0.657, your value = 0.657
f3: true value = 2.372, your value = 2.372
like original = 0.139, like from forward pass = 0.139


**Шаг 2:** Вычислим производные от $\ell_i$ относительно промежуточных величин, которые мы только что вычислили, но в обратном порядке:

\begin{align}
\quad \frac{\partial \ell_i}{\partial f_3}, \quad \frac{\partial \ell_i}{\partial h_3}, \quad \frac{\partial \ell_i}{\partial f_2}, \quad
\frac{\partial \ell_i}{\partial h_2}, \quad \frac{\partial \ell_i}{\partial f_1}, \quad \frac{\partial \ell_i}{\partial h_1},  \quad\text{и} \quad \frac{\partial \ell_i}{\partial f_0}.
\end{align}


Первая из этих производных проста:

\begin{equation}
\frac{\partial \ell_i}{\partial f_{3}} = 2 (f_3-y).
\end{equation}


Другая производная может быть вычислена с помощью цепного правила:

\begin{equation}
\frac{\partial \ell_i}{\partial h_{3}} =\frac{\partial f_{3}}{\partial h_{3}} \frac{\partial \ell_i}{\partial f_{3}} .
\end{equation}


В левой части спрашивается, как изменяется $\ell_i$ при изменении $h_{3}$.  В правой части говорится, что мы можем разложить это на то, как $\ell_i$ изменяется при изменении $f_{3}$ и как $f_{3}$ изменяется при изменении $h_{3}$.  Таким образом, вы получаете цепочку происходящих событий: $h_{3}$ изменяет $f_{3}$, что изменяет $\ell_i$, а производные представляют эффекты этой цепочки.  Обратите внимание, что мы уже вычислили первую из этих производных, и она равна $2 (f_3-y)$. Мы вычислили $f_{3}$ на шаге 1.  Второй член является производной от $\beta_{3} + \omega_{3}h_{3}$ по отношению к $h_3$, который является просто $\omega_3$.


Мы можем продолжить в том же духе, вычисляя производные выходных данных по отношению к этим промежуточным величинам:

\begin{align}
\frac{\partial \ell_i}{\partial f_{2}} &=& \frac{\partial h_{3}}{\partial f_{2}}\left(
\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)
\nonumber \\
\frac{\partial \ell_i}{\partial h_{2}} &=& \frac{\partial f_{2}}{\partial h_{2}}\left(\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}}\right)\nonumber \\
\frac{\partial \ell_i}{\partial f_{1}} &=& \frac{\partial h_{2}}{\partial f_{1}}\left( \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)\nonumber \\
\frac{\partial \ell_i}{\partial h_{1}} &=& \frac{\partial f_{1}}{\partial h_{1}}\left(\frac{\partial h_{2}}{\partial f_{1}} \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)\nonumber \\
\frac{\partial \ell_i}{\partial f_{0}} &=& \frac{\partial h_{1}}{\partial f_{0}}\left(\frac{\partial f_{1}}{\partial h_{1}}\frac{\partial h_{2}}{\partial f_{1}} \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right).
\end{align}


В каждом случае мы уже вычислили все члены, кроме последнего, на предыдущем шаге, и последний член вычислить просто.  Это называется **обратным проходом** (**backward pass**).

In [9]:
# TODO - Вычислим производные выходных данных относительно
# промежуточных вычислений h_k и f_k (т.е. выполним обратный проход)
# Первые два уже выполнены за вас.
dldf3 = 2* (f3 - y)
dldh3 = omega3 * dldf3
# Замените приведенный ниже код
dldf2 = -1*np.sin(f2)*dldh3
dldh2 = omega2*dldf2
dldf1 = np.exp(f1)*dldh2
dldh1 = omega1*dldf1
dldf0 = np.cos(f0)*dldh1


In [10]:
# Давайте проверим, правильно ли мы их посчитали
print("dldf3: true value = %3.3f, your value = %3.3f"%(0.745, dldf3))
print("dldh3: true value = %3.3f, your value = %3.3f"%(2.234, dldh3))
print("dldf2: true value = %3.3f, your value = %3.3f"%(-1.683, dldf2))
print("dldh2: true value = %3.3f, your value = %3.3f"%(-3.366, dldh2))
print("dldf1: true value = %3.3f, your value = %3.3f"%(-17.060, dldf1))
print("dldh1: true value = %3.3f, your value = %3.3f"%(6.824, dldh1))
print("dldf0: true value = %3.3f, your value = %3.3f"%(2.281, dldf0))

dldf3: true value = 0.745, your value = 0.745
dldh3: true value = 2.234, your value = 2.234
dldf2: true value = -1.683, your value = -1.683
dldh2: true value = -3.366, your value = -3.366
dldf1: true value = -17.060, your value = -17.060
dldh1: true value = 6.824, your value = 6.824
dldf0: true value = 2.281, your value = 2.281


In [11]:
# TODO - Вычислим конечные производные по отношению к членам бета и омега
dldbeta3 = dldf3
dldomega3 = dldf3*h3
dldbeta2 = dldf2
dldomega2 = dldf2*h2
dldbeta1 = dldf1
dldomega1 = dldf1*h1
dldbeta0 = dldf0
dldomega0 = dldf0*x


In [12]:
# Давайте проверим, правильно ли мы их посчитали
print('dldbeta3: Your value = %3.3f, True value = %3.3f'%(dldbeta3, 0.745))
print('dldomega3: Your value = %3.3f, True value = %3.3f'%(dldomega3, 0.489))
print('dldbeta2: Your value = %3.3f, True value = %3.3f'%(dldbeta2, -1.683))
print('dldomega2: Your value = %3.3f, True value = %3.3f'%(dldomega2, -8.530))
print('dldbeta1: Your value = %3.3f, True value = %3.3f'%(dldbeta1, -17.060))
print('dldomega1: Your value = %3.3f, True value = %3.3f'%(dldomega1, -16.079))
print('dldbeta0: Your value = %3.3f, True value = %3.3f'%(dldbeta0, 2.281))
print('dldomega0: Your value = %3.3f, Function value = %3.3f, Finite difference value = %3.3f'%(dldomega0, dldomega0_func, dldomega0_fd))

dldbeta3: Your value = 0.745, True value = 0.745
dldomega3: Your value = 0.489, True value = 0.489
dldbeta2: Your value = -1.683, True value = -1.683
dldomega2: Your value = -8.530, True value = -8.530
dldbeta1: Your value = -17.060, True value = -17.060
dldomega1: Your value = -16.079, True value = -16.079
dldbeta0: Your value = 2.281, True value = 2.281
dldomega0: Your value = 5.246, Function value = 5.246, Finite difference value = 5.246


Используя этот метод, мы можем довольно легко вычислить производные, не прибегая к вычислению очень сложных выражений.  В следующем разделе мы применим этот же метод к глубокой нейронной сети.